# Stable Diffusion

text-to-image generator

official github: https://github.com/CompVis/stable-diffusion

notebook copied from https://github.com/woctezuma/stable-diffusion-colab

In [ ]:
!nvidia-smi

In [ ]:
%pip install --quiet --upgrade diffusers transformers scipy mediapy

In [ ]:
import torch, mediapy
from diffusers import (
	StableDiffusionPipeline, StableDiffusionImg2ImgPipeline, StableDiffusionInpaintPipeline,
	LMSDiscreteScheduler, # sampler suggested by DreamStudio
	PNDMScheduler, # default in v1
	DDIMScheduler, # default in v2, great results at a blazing fast speed
	EulerDiscreteScheduler, EulerAncestralDiscreteScheduler, # changes generation style a lot at each step
	DPMSolverMultistepScheduler # fastest
)

safety check removed

In [ ]:
model_id = "stabilityai/stable-diffusion-2-base"
pipe = StableDiffusionPipeline.from_pretrained(
	model_id, scheduler=DDIMScheduler.from_pretrained(model_id, subfolder="scheduler"),
	revision="fp16", torch_dtype=torch.float16, safety_checker=None
).to("cuda")

In [ ]:
images = pipe(
	prompt="a photo of an astronaut riding a horse on mars",
	# negative_prompt="",
	# height=512, width=512,
	num_images_per_prompt=5,
	guidance_scale=7.5, # should be between 0 and 20
	num_inference_steps=50 # should be between 10 and 150
).images

mediapy.show_images(images)
# images[0].save("output.jpg")